# DM (device manufacturer) connects to music (device) and assign them ownership - Music

## 3. Initialize Music

In [1]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"
API_KEY = "bob_api_123456789"

agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key=API_KEY)

agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)


######################
# Remember to update #
######################

invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '2065f3a1-1568-41ad-9a0d-d217d3eecfd0', 'recipientKeys': ['2EZgL2gMffbxQFWBS2mVkuLmFPAzaMVgwGZXTswYCE6y'], 'serviceEndpoint': 'http://192.168.65.3:8020', 'label': 'Alice'}


# Receive Invitation
response = await agent_controller.connections.accept_connection(invitation)
# Print out accepted Invite and Alice's connection ID
print("Connection", response)
alice_id = response["connection_id"]


IPython autoawait is `on`, and set to use `asyncio`
Subscribing too: connections
Connection Handler Called
Connection 6e50c915-b645-4983-aa62-9b743915bf84 in State invitation
Connection Handler Called
Connection 6e50c915-b645-4983-aa62-9b743915bf84 in State request
Connection {'rfc23_state': 'request-sent', 'state': 'request', 'their_label': 'Alice', 'their_role': 'inviter', 'invitation_key': '2EZgL2gMffbxQFWBS2mVkuLmFPAzaMVgwGZXTswYCE6y', 'invitation_mode': 'once', 'created_at': '2023-01-03 12:11:05.203200Z', 'request_id': '3c5e3b3e-38ae-4df4-85b7-644a9c550a09', 'connection_id': '6e50c915-b645-4983-aa62-9b743915bf84', 'routing_state': 'none', 'updated_at': '2023-01-03 12:11:05.262616Z', 'accept': 'manual', 'my_did': 'JwZFj9aDdiL1d4pJSSL3Wj'}
Connection Handler Called
Connection 6e50c915-b645-4983-aa62-9b743915bf84 in State response
Connection Handler Called
Connection 6e50c915-b645-4983-aa62-9b743915bf84 in State active


In [2]:
# Print connection list
connection = await agent_controller.connections.get_connection(alice_id)
print("Alice AGENT CONNECTION")
print(connection)
print("State:", connection["state"])

Alice AGENT CONNECTION
{'rfc23_state': 'completed', 'state': 'active', 'their_label': 'Alice', 'their_role': 'inviter', 'invitation_key': '2EZgL2gMffbxQFWBS2mVkuLmFPAzaMVgwGZXTswYCE6y', 'invitation_mode': 'once', 'created_at': '2023-01-03 12:11:05.203200Z', 'request_id': '3c5e3b3e-38ae-4df4-85b7-644a9c550a09', 'their_did': 'XtNBn5VgqaouerjBv9zGiy', 'connection_id': '6e50c915-b645-4983-aa62-9b743915bf84', 'routing_state': 'none', 'updated_at': '2023-01-03 12:11:59.345427Z', 'accept': 'manual', 'my_did': 'JwZFj9aDdiL1d4pJSSL3Wj'}
State: active


# Give MUSIC the VC of ownership and origin - DM

## 3. Credential Exchange!

In [3]:
response = await agent_controller.issuer.get_records()
results = response["results"]
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")

Credential exchange 0a9a0112-9a00-4449-8894-f80fe533e426, role: holder, state: offer_received
Being offered: [{'name': 'author', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'name', 'value': 'balabababa music'}, {'name': 'time', 'value': '01-03-2023, 12:12:33'}, {'name': 'origin', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'owner', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'version', 'value': 'v1'}]


In [4]:
# Request credential from issuer
record = await agent_controller.issuer.send_request_for_record(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange 0a9a0112-9a00-4449-8894-f80fe533e426, role: holder, state: request_sent


In [6]:
record = await agent_controller.issuer.get_record_by_id(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange 0a9a0112-9a00-4449-8894-f80fe533e426, role: holder, state: credential_received


In [7]:
# Store credential
response = await agent_controller.issuer.store_credential(cred_ex_id, "My OM Credential")
state = response['state']
role = response['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange 0a9a0112-9a00-4449-8894-f80fe533e426, role: holder, state: credential_acked


## Remove connection !!!

In [8]:
remove = await agent_controller.connections.remove_connection(alice_id)
print(remove)
response = await agent_controller.connections.get_connections()
print(response)

{}
{'results': []}


In [9]:
response = await agent_controller.terminate()
print(response)

None
